# Neural Network Implementation
We finally approach the neural network implementation of this fucking paper

In [111]:
# Packages
import numpy as np
import pandas as pd
import torch
import scipy.stats as stats
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

# All Required DataSets
IVSurface = np.load('testSurface.pkl', allow_pickle = True)
SP500 = np.load('SP500.pkl', allow_pickle = True)
VIX = np.load('VIX.pkl', allow_pickle = True)

In [112]:
mergedData = pd.merge(pd.merge(IVSurface, SP500, how = 'left', on = 'Date'), VIX, how = 'left', on = 'Date')
mergedData = mergedData.filter(['strike', 'daysExp', 'fittedIV', 'Returns', 'VIX'])
mergedData['IVChange'] = mergedData.fittedIV.diff(periods = 441)
mergedData = mergedData.shift(periods = -441).dropna()
mergedData.to_pickle('finalDataSet.pkl')

## Preparing Dataloader for PyTorch

In [149]:
class Factor_Model_Dataloader():
    def __init__(self, X, y, scale_data = True):
        if scale_data:
            X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]

## Creating Neural Network

In [150]:
class Four_Factor_Model(nn.Module):
    '''Multilayer Feed Forward Network for regression'''
    def __init__(self, hiddenNodes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(4,hiddenNodes),
            nn.ReLU(),
            nn.Linear(hiddenNodes, hiddenNodes),
            nn.ReLU(),
            nn.Linear(hiddenNodes, 1)
        )
        
    def forward(self, x):
        return self.layers(x)

## Preparing Dataset

In [151]:
[X, y] = [mergedData.filter(['strike', 'daysExp', 'Returns', 'VIX']).values,
          mergedData.filter(['IVChange']).values]

torchTrainingSet = Factor_Model_Dataloader(X, y)
trainLoader = torch.utils.data.DataLoader(torchTrainingSet, batch_size = 100, shuffle = True, num_workers = 0)


## Choosing Loss Functions

In [152]:
four_factor_model = Four_Factor_Model(100)
loss_func = nn.MSELoss()
opt = torch.optim.Adam(four_factor_model.parameters(), lr = 1e-4)

## Training Neural Network

In [155]:
epochs = 100
for epoch in range(0, epochs):
    print(f'Starting epoch {epoch + 1}')
    currentLoss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        
        opt.zero_grad()
        outputs = four_factor_model(inputs)
        loss = loss_func(outputs, targets)
        loss.backward()
        opt.step()
        currentLoss += loss.item()
        
        if i % 100 == 0:
            print('Loss after mini-batch %5d: %.5f' % (i + 1, currentLoss))
            current_loss = 0.0
            
print('Training finished')

Starting epoch 1
Loss after mini-batch     1: 0.00012
Starting epoch 2
Loss after mini-batch     1: 0.00014
Starting epoch 3
Loss after mini-batch     1: 0.00011
Starting epoch 4
Loss after mini-batch     1: 0.00012
Starting epoch 5
Loss after mini-batch     1: 0.00018
Starting epoch 6
Loss after mini-batch     1: 0.00014
Starting epoch 7
Loss after mini-batch     1: 0.00016
Starting epoch 8
Loss after mini-batch     1: 0.00008
Starting epoch 9
Loss after mini-batch     1: 0.00018
Starting epoch 10
Loss after mini-batch     1: 0.00014
Starting epoch 11
Loss after mini-batch     1: 0.00015
Starting epoch 12
Loss after mini-batch     1: 0.00018
Starting epoch 13
Loss after mini-batch     1: 0.00009
Starting epoch 14
Loss after mini-batch     1: 0.00014
Starting epoch 15
Loss after mini-batch     1: 0.00009
Starting epoch 16
Loss after mini-batch     1: 0.00010
Starting epoch 17
Loss after mini-batch     1: 0.00011
Starting epoch 18
Loss after mini-batch     1: 0.00006
Starting epoch 19
L

In [156]:
pd.DataFrame(np.stack([four_factor_model(inputs).detach().numpy(), targets], axis = 0)[:,:,0].T)

,0,1
0,0.006617,0.009933
1,0.001646,-0.003127
2,-0.055889,-0.054539
3,-0.006336,-0.005135
4,0.015197,0.015826
...,...,...
74,-0.024872,-0.027833
75,0.017329,0.016825
76,-0.009951,-0.012147
77,0.002100,-0.001728
